In [1]:
print("hello world")

hello world


In [6]:
import plotly

plotly.offline.init_notebook_mode(connected=True)
plotly.__version__

'2.2.3'

In [8]:
from plotly.graph_objs import Scatter, Layout

plotly.offline.iplot({
    "data": [Scatter(x=[1, 2, 3, 4], y=[4, 3, 2, 1])],
    "layout": Layout(title="hello world")
})